# Mojo 0.6
This is a test notebook that explores some of the new features in [Mojo 0.6 release](https://docs.modular.com/mojo/changelog.html?utm_content=273987722&utm_medium=social&utm_source=twitter&hss_channel=tw-1483918307484848132)

In [29]:
@value
struct test: 
  fn sayhello(self):
    print("Hello")

let t = test()
t.sayhello()

Hello


## Movable, Copyable

New traits added for move and copy constructors that are required for types that can be stored in a DynamicVector.

In [30]:

struct van(Movable, Copyable, CollectionElement):
  var thing: Int

  fn __init__(inout self):
    print("van init")
    self.thing = 0

  fn __moveinit__(inout self: van, owned existing: van ):
    print("van moveinit")
    self.thing = existing.thing

  fn __copyinit__(inout self: van, existing: van ):
    print("van copyinit")
    self.thing = existing.thing

  fn __del__(owned self):
    print("del")

  
fn testMovableCopyable():
   print("testMovableCopyable")
   var vans: DynamicVector[van] = DynamicVector[van]() 
   vans.append(van())
   print("len vans",len(vans))
   print("Done")
   # surprising del before
   # explict calling del on vans weird also

## Partial automatic parametarization

Readme says: "when a function is declared with an argument of a partially bound type, the unbound parameters of that type are implicitly added to the function’s input parameters."

This assumes knowledge of what a partially bound type is.  What is that?

From the documentation: [type bindings](https://docs.modular.com/mojo/manual/parameters/#fully-bound-partially-bound-and-unbound-types)

In [5]:
struct diary[storetype: CollectionElement,maxlength: Int, maxentries: Int]:
   var storage: DynamicVector[storetype]

   fn __init__(inout self):
      self.storage = DynamicVector[storetype]()

# when we use the diary type, we supply / bind a parameter
var recorder: diary[String,1,1] = diary[String,1,1]()

# function has a unbound form of diary as function argument
fn partiallyboundfunction(thediary: diary):
   pass

# calling funciton performas the parameter binding
partiallyboundfunction(diary[String,1,1]())

## Explicit unbound parameters

Referencing a parameterized type in a function call supplying `_` as a parameter makes it possible to disambiguate which parameters are supplied as arguments by the caller: 

The readme implies that if values are supplied for some parameters with others explicitly unbound via _, only the unbound ones need to be provided when calling.  This doesn't seem to work in the following test.

In [6]:
fn explicitlyunbound(thediary: diary[String, _, 10]):
    pass

explicitlyunbound(diary[1]())


error: Expression [6]:1:37: 'diary' expects 1 input parameter, but 3 were specified
fn explicitlyunbound(thediary: diary[String, _, 10]):
                                    ^

Expression [5]:1:1: 'diary' declared here
struct diary[storetype: CollectionElement]:
^

expression failed to parse (no further compiler diagnostics)

## Partial type binding in alias definitions
Create an alias with partial binding, supply remaining parameters against alias

In [7]:

alias defineddiary = diary[String,1,2]
var dd: defineddiary = defineddiary()

alias stringdiary = diary[String,_,_]
var sd: stringdiary[1,1] = stringdiary[1,1]()

alias tendiary = diary[_,10,_]
var td: tendiary[String,10]

error: Expression [7]:2:27: 'diary' expects 1 input parameter, but 3 were specified
alias defineddiary = diary[String,1,2]
                          ^

Expression [5]:1:1: 'diary' declared here
struct diary[storetype: CollectionElement]:
^

error: Expression [7]:5:26: 'diary' expects 1 input parameter, but 3 were specified
alias stringdiary = diary[String,_,_]
                         ^

Expression [5]:1:1: 'diary' declared here
struct diary[storetype: CollectionElement]:
^

error: Expression [7]:8:23: 'diary' expects 1 input parameter, but 3 were specified
alias tendiary = diary[_,10,_]
                      ^

Expression [5]:1:1: 'diary' declared here
struct diary[storetype: CollectionElement]:
^

expression failed to parse (no further compiler diagnostics)

## __mlir_op supports operations that return multiple Items

This is quite advanced and requires further investigation

In [8]:
var a: Tuple[String, Int, Int] = Tuple(String("Hello"), 10, 10)

struct multiresult:

   fn test(self):
        # The `ret` variable has type `Tuple[Int, Int]`.
        let ret = __mlir_op.`multi_result_op`[ _type = (Int, Int) ]()


error: Expression [8]:7:68: use of unregistered MLIR operation 'multi_result_op'
        let ret = __mlir_op.`multi_result_op`[ _type = (Int, Int) ]()
                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^~

expression failed to parse (no further compiler diagnostics)

## Reading raw bytes from a file
This just works even in a mojo notebook.

Note the use of seek to reset the stream read position

In [17]:
with open("LICENSE", "r") as f:
    data = f.read_bytes()
    print("Bytes:", data.bytecount())

    f.seek(0)
    data2 = f.read_bytes(1000)
    print("Bytes after seek:", data2.bytecount())

Bytes: 11357
Bytes after seek: 1000


## Path improvements


In [3]:
let readmepath = Path("README.md")
let text = readmepath.read_text()
print(text)

let blob = readmepath.read_bytes()
let typeofblob = print("In mojo everything is a tensor:", blob.shape())

Hello! World!
In mojo everything is a tensor: 13


## Tensor Load and Save
Look Ma, I can easily load and save a Tensor

In [2]:
from random import rand
from tensor import Tensor, TensorSpec, TensorShape
from utils.index import Index

let test = rand[DType.float32](10,10,3)
let spec = TensorSpec(DType.float32, 10, 10, 3)
var tensor = Tensor[DType.float32](spec)
# i'm sure there is a better way of doing this
for i in range(10):
    for j in range(10):
        for k in range(3):
            tensor[Index(i,j,k)] = 0.1
print(tensor)
tensor.save("./cool")

Tensor([[[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
..., 
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612]],
[[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
..., 
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612]],
[[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],
[0.10000000149011612, 0.10000000149011612, 0.10000000149011612],


In [6]:
from tensor import Tensor, TensorSpec, TensorShape

let cooltensor = Tensor[DType.float32].load("./cool")
print(cooltensor)

Tensor([[[0.1315377950668335, 0.458650141954422, 0.0],
[0.67886471748352051, 0.93469291925430298, 0.0],
[0.034572109580039978, 0.52970021963119507, 0.0],
..., 
[0.75335586071014404, 0.072685882449150085, 0.0],
[0.43641141057014465, 0.47773176431655884, 0.0],
[0.0, 0.0, 0.0]],
[[0.50452291965484619, 0.3190329372882843, 0.0],
[0.090732894837856293, 0.073749072849750519, 0.0],
[0.91381746530532837, 0.46444582939147949, 0.0],
..., 
[0.17832770943641663, 0.5716547966003418, 0.0],
[0.49848011136054993, 0.74829262495040894, 0.0],
[0.0, 0.0, 0.0]],
[[0.27458813786506653, 0.41429325938224792, 0.0],
[0.23991081118583679, 0.31753954291343689, 0.0],
[0.68134623765945435, 0.38772532343864441, 0.0],
..., 
[0.20325033366680145, 0.90167349576950073, 0.0],
[0.41031304001808167, 0.88564836978912354, 0.0],
[0.0, 0.0, 0.0]]
..., ,
[[0.47780430316925049, 0.51998847723007202, 0.0],
[0.83368211984634399, 0.57777601480484009, 0.0],
[0.98688298463821411, 0.0016574158798903227, 0.0],
..., 
[0.76061731576919556,

## Pointer subscripting

In [2]:
let dtp = DTypePointer[DType.float32]().alloc(10)
let p = Pointer[Float32].alloc(10)
for i in range(10):
    let fi = Float32(i)
    p[i] = fi

    # is a Dtype.float32 same as a Float32 in terms of memory layout?
    # what is the difference between DType.float32 and Float32?
    dtp[i] = fi

    # how do you create a DType.float32?
    # let fi2 = DType[DType.float32](i)
    # let dtp[i] = fi2


## String enhancements
`rfind` (last occurance of a substring) and `split` are added.

In [3]:
# string can be split, not inplace obviously
let foo: String = "a string in another string"
let splitstring = foo.split(" ")
print(len(splitstring))

let finding = foo.rfind("in")
print(finding)

# stringliteral cannot
let bar = "another string"
# bar.split(" ")

5
23


## Variadic Args
Using * in front of an argument in a function indicates a variadic which can take variable number of arguments.  In mojo 0.6 this is iterable

In [1]:
fn print_ints(*nums: Int):
    for num in nums:
        print(num)
    print(len(nums))

print_ints(1,2,3,4,5)

1
2
3
4
5
5


## Testing package

In [1]:
import testing
testing.assert_true(True==False,"true!=false")

Error: AssertionError: true!=false


## Benchmark

In [4]:
import benchmark

fn timer():
    pass

let report = benchmark.run[timer]()
report.print(benchmark.Unit.ms)

---------------------
Benchmark Report (ms)
---------------------
Mean: 2.8023292368583394e-14
Total: 222.817261
Iters: 7951145000000000
Warmup Mean: 2.0000000000000002e-05
Warmup Total: 4.0000000000000003e-05
Warmup Iters: 2
Fastest Mean: 1.9000000000000001e-14
Slowest Mean: 1.5284700000000002e-10

